## Model Selection

This notebook should include preliminary and baseline modeling.
- Try as many different models as possible.
- Don't worry about hyperparameter tuning or cross validation here.
- Ideas include:
    - linear regression
    - support vector machines
    - random forest
    - xgboost

In [1]:
import os
import sys
import logging

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import StandardScaler
from functions_variables import *
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Load the datasets
X_train = pd.read_csv('../data/processed/X_train.csv')
X_test = pd.read_csv('../data/processed/X_test.csv')
Y_train = pd.read_csv('../data/processed/Y_train.csv').squeeze()
Y_test = pd.read_csv('../data/processed/Y_test.csv').squeeze()


In [3]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", Y_train.shape)
print("y_test shape:", Y_test.shape)

X_train shape: (4511, 22)
X_test shape: (1128, 22)
y_train shape: (4511,)
y_test shape: (1128,)


In [14]:
scaler_features = StandardScaler()

columns_to_scale = [
    'description_baths', 'description_beds', 'description_garage',
    'description_sqft', 'description_stories', 'description_year_built',
    'year_sold', 'year_listed', 'days_on_market', 'city_frequency', 'state_frequency'
]

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[columns_to_scale] = scaler_features.fit_transform(X_train[columns_to_scale])
X_test_scaled[columns_to_scale] = scaler_features.transform(X_test[columns_to_scale])

scaler_target = StandardScaler()
y_train_scaled = scaler_target.fit_transform(Y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler_target.transform(Y_test.values.reshape(-1, 1)).flatten()



In [5]:
processed_data_path = '../data/processed/'
X_train_scaled.to_csv(processed_data_path + 'X_train_scaled.csv', index=False)
X_test_scaled.to_csv(processed_data_path + 'X_test_scaled.csv', index=False)

# Save y_train_scaled and y_test_scaled
pd.DataFrame(y_train_scaled, columns=["y_train_scaled"]).to_csv(processed_data_path + 'y_train_scaled.csv', index=False)
pd.DataFrame(y_test_scaled, columns=["y_test_scaled"]).to_csv(processed_data_path + 'y_test_scaled.csv', index=False)

In [15]:
models = {
    'Linear Regression': LinearRegression(),
    'SVR': SVR(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

results = {}

for model_name, model in models.items():
    results[model_name] = evaluate_model(model, X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, scaler_target)

# Display results
for model_name, metrics in results.items():
    print(f"\nModel: {model_name}")
    for metric_name, metric_value in metrics.items():
        print(f"  {metric_name}: {metric_value:.2f}")

LinearRegression:
  Train RMSE: $218220.76, Test RMSE: $225634.32
  Train MAE: $139385.10, Test MAE: $145045.50
  Train R^2: 0.43, Test R^2: 0.47
SVR:
  Train RMSE: $129333.31, Test RMSE: $151565.00
  Train MAE: $58366.07, Test MAE: $72227.78
  Train R^2: 0.80, Test R^2: 0.76
RandomForestRegressor:
  Train RMSE: $16560.35, Test RMSE: $40315.56
  Train MAE: $4951.29, Test MAE: $11727.67
  Train R^2: 1.00, Test R^2: 0.98
XGBRegressor:
  Train RMSE: $12616.08, Test RMSE: $37015.39
  Train MAE: $8478.83, Test MAE: $14151.74
  Train R^2: 1.00, Test R^2: 0.99

Model: Linear Regression
  Train RMSE: 218220.76
  Test RMSE: 225634.32
  Train MAE: 139385.10
  Test MAE: 145045.50
  Train R^2: 0.43
  Test R^2: 0.47

Model: SVR
  Train RMSE: 129333.31
  Test RMSE: 151565.00
  Train MAE: 58366.07
  Test MAE: 72227.78
  Train R^2: 0.80
  Test R^2: 0.76

Model: Random Forest
  Train RMSE: 16560.35
  Test RMSE: 40315.56
  Train MAE: 4951.29
  Test MAE: 11727.67
  Train R^2: 1.00
  Test R^2: 0.98

Model

Consider what metrics you want to use to evaluate success.
- If you think about mean squared error, can we actually relate to the amount of error?
- Try root mean squared error so that error is closer to the original units (dollars)
- What does RMSE do to outliers?
- Is mean absolute error a good metric for this problem?
- What about R^2? Adjusted R^2?
- Briefly describe your reasons for picking the metrics you use

In [6]:
# gather evaluation metrics and compare results

## Feature Selection - STRETCH

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its recommended you complete it if you have time!**

Even with all the preprocessing we did in Notebook 1, you probably still have a lot of features. Are they all important for prediction?

Investigate some feature selection algorithms (Lasso, RFE, Forward/Backward Selection)
- Perform feature selection to get a reduced subset of your original features
- Refit your models with this reduced dimensionality - how does performance change on your chosen metrics?
- Based on this, should you include feature selection in your final pipeline? Explain

Remember, feature selection often doesn't directly improve performance, but if performance remains the same, a simpler model is often preferrable. 



In [7]:
# Feature selection
base_model = LinearRegression()

forward_selector = SequentialFeatureSelector(
    estimator=base_model,
    n_features_to_select=5,  
    direction='forward',  
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1
)

forward_selector.fit(X_train_scaled, y_train_scaled)

selected_features_forward = X_train_scaled.columns[forward_selector.get_support()]
print("Selected Features (Forward Selection):", selected_features_forward)

X_train_forward = X_train_scaled[selected_features_forward]
X_test_forward = X_test_scaled[selected_features_forward]

Selected Features (Forward Selection): Index(['description_baths', 'description_sqft', 'community_security_features',
       'view', 'city_view'],
      dtype='object')


In [8]:
# Evaluate Linear Regression on forward-selected features
lr_model = LinearRegression()
results_forward = evaluate_model(lr_model, X_train_forward, X_test_forward, y_train_scaled, y_test_scaled, scaler_target)
print("\nResults with Forward-Selected Features:")
for metric, value in results_forward.items():
    print(f"{metric}: {value:.2f}")

LinearRegression:
  Train RMSE: $227451.54, Test RMSE: $236042.89
  Train MAE: $143682.38, Test MAE: $150038.76
  Train R^2: 0.38, Test R^2: 0.42

Results with Forward-Selected Features:
Train RMSE: 227451.54
Test RMSE: 236042.89
Train MAE: 143682.38
Test MAE: 150038.76
Train R^2: 0.38
Test R^2: 0.42


In [9]:
# Perform backward feature selection
backward_selector = SequentialFeatureSelector(
    estimator=base_model,
    n_features_to_select=5,  # Specify the desired number of features
    direction='backward',  # Backward selection
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1
)

# Fit the selector on the training data
backward_selector.fit(X_train_scaled, y_train_scaled)

# Get the selected features
selected_features_backward = X_train_scaled.columns[backward_selector.get_support()]
print("Selected Features (Backward Selection):", selected_features_backward)

# Create new datasets with selected features
X_train_backward = X_train_scaled[selected_features_backward]
X_test_backward = X_test_scaled[selected_features_backward]

Selected Features (Backward Selection): Index(['description_baths', 'description_sqft', 'community_security_features',
       'view', 'city_view'],
      dtype='object')


In [10]:
# Evaluate Linear Regression on backward-selected features
results_backward = evaluate_model(lr_model, X_train_backward, X_test_backward, y_train_scaled, y_test_scaled, scaler_target)
print("\nResults with Backward-Selected Features:")
for metric, value in results_backward.items():
    print(f"{metric}: {value:.2f}")

LinearRegression:
  Train RMSE: $227451.54, Test RMSE: $236042.89
  Train MAE: $143682.38, Test MAE: $150038.76
  Train R^2: 0.38, Test R^2: 0.42

Results with Backward-Selected Features:
Train RMSE: 227451.54
Test RMSE: 236042.89
Train MAE: 143682.38
Test MAE: 150038.76
Train R^2: 0.38
Test R^2: 0.42


In [11]:
from sklearn.feature_selection import RFE

# Update SVR to use linear kernel
models = {
    'Linear Regression': LinearRegression(),
    'SVR (Linear Kernel)': SVR(kernel='linear'),
    'Random Forest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
}

results = {}

# Loop through each model
for model_name, model in models.items():
    print(f"Evaluating {model_name} with RFE...")
    
    # Apply RFE (skip SVR with non-linear kernel)
    try:
        rfe = RFE(estimator=model, n_features_to_select=15)  # Adjust the number of features to select
        X_train_rfe = rfe.fit_transform(X_train_scaled, y_train_scaled)
        X_test_rfe = rfe.transform(X_test_scaled)
        
        # Evaluate the model with the reduced feature set
        results[model_name] = evaluate_model(model, X_train_rfe, X_test_rfe, y_train_scaled, y_test_scaled, scaler_target)
    except ValueError as e:
        print(f"Skipping RFE for {model_name}: {e}")
        continue

# Display results
for model_name, metrics in results.items():
    print(f"\nModel: {model_name}")
    for metric_name, metric_value in metrics.items():
        print(f"  {metric_name}: {metric_value:.2f}")

Evaluating Linear Regression with RFE...
LinearRegression:
  Train RMSE: $218827.46, Test RMSE: $226154.03
  Train MAE: $138799.48, Test MAE: $144515.44
  Train R^2: 0.43, Test R^2: 0.47
Evaluating SVR (Linear Kernel) with RFE...
SVR:
  Train RMSE: $229750.83, Test RMSE: $240414.21
  Train MAE: $129831.58, Test MAE: $137087.51
  Train R^2: 0.37, Test R^2: 0.40
Evaluating Random Forest with RFE...
RandomForestRegressor:
  Train RMSE: $16728.11, Test RMSE: $39179.02
  Train MAE: $4939.54, Test MAE: $11685.11
  Train R^2: 1.00, Test R^2: 0.98
Evaluating XGBoost with RFE...
XGBRegressor:
  Train RMSE: $15590.07, Test RMSE: $28820.07
  Train MAE: $10447.51, Test MAE: $14733.02
  Train R^2: 1.00, Test R^2: 0.99

Model: Linear Regression
  Train RMSE: 218827.46
  Test RMSE: 226154.03
  Train MAE: 138799.48
  Test MAE: 144515.44
  Train R^2: 0.43
  Test R^2: 0.47

Model: SVR (Linear Kernel)
  Train RMSE: 229750.83
  Test RMSE: 240414.21
  Train MAE: 129831.58
  Test MAE: 137087.51
  Train R^2:

In [12]:
from sklearn.linear_model import Lasso
# Define the Lasso Regression model
lasso_model = Lasso()

# Define the hyperparameter grid
lasso_param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10]  # Regularization strength
}

# Set up GridSearchCV
lasso_grid_search = GridSearchCV(
    estimator=lasso_model,
    param_grid=lasso_param_grid,
    cv=5,
    scoring='neg_root_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

# Fit the model
lasso_grid_search.fit(X_train_scaled, y_train_scaled)

# Best parameters and performance
print("Best Parameters (Lasso):", lasso_grid_search.best_params_)
print("Best RMSE (Lasso):", -lasso_grid_search.best_score_)

# Evaluate the model on the train and test sets
best_lasso_model = lasso_grid_search.best_estimator_
results_lasso = evaluate_model(best_lasso_model, X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, scaler_target)

# Display evaluation results
for metric, value in results_lasso.items():
    print(f"{metric}: {value:.2f}")

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters (Lasso): {'alpha': 0.001}
Best RMSE (Lasso): 0.7609943955509639
Lasso:
  Train RMSE: $218240.62, Test RMSE: $225650.17
  Train MAE: $139107.88, Test MAE: $144784.54
  Train R^2: 0.43, Test R^2: 0.47
Train RMSE: 218240.62
Test RMSE: 225650.17
Train MAE: 139107.88
Test MAE: 144784.54
Train R^2: 0.43
Test R^2: 0.47


In [13]:
# gather evaluation metrics and compare to the previous step (full feature set)